# 340_Join_FIFA_Fantasy1314 
## Purpose 
In this notebook we read both our Fantasty and FIFA cleaned datasets and join them and performing any necessary cleaning.  
## Datasets 
* _Input_: FIFA18.csv,Fantasy1314.csv
* _Output_: Joined1314.csv

In [1]:
import os.path
import numpy as np
import pandas as pd

### Reading in our cleaned FIFA and Fantasy datasets.

In [2]:
Fifa = pd.read_csv("../../data/prep/FIFA18.csv")
Fantasy1314 = pd.read_csv("../../data/prep/Fantasy1314.csv")

### Merging the two datasets on Surname and First Initial
* Use pd.merge to join the tables  .
* Drop irrelevant column 'Unnamed: 0_x'.

In [3]:
Join1314 = pd.merge(Fifa, Fantasy1314, on=['Surname','FirstInitial'])
Join1314.drop(Join1314.columns[[0]], axis=1, inplace=True)

In [4]:
Join1314.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera Division,1985-02-05,185.0,...,0,3,0,0,0,0,0,30,7.733333,Cristiano
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera Division,1987-06-24,170.0,...,2,4,0,1,0,0,0,31,7.032258,Lionel
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,...,7,11,0,0,0,0,0,26,4.153846,Neymar
3,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,0,3,0,0,0,0,0,33,7.696970,Luis
4,225591,L. Suarez,Leonardo Suarez,Villarreal CF,https://cdn.sofifa.org/18/teams/483.png,1673,21,Spanish Primera Division,1996-03-30,167.0,...,0,3,0,0,0,0,0,33,7.696970,Luis


### Creating a while loop to remove players
* Removes all players thats Firstname is not contained in 'name'
* Removes all players thats Firstname is not contained in 'full_name'.

In [5]:
i = 0
array = []

while i < len(Join1314):
    if Join1314['Firstname'][i] in Join1314['name'][i] or Join1314['Firstname'][i] in Join1314['full_name'][i]:
        array.append(True)
        i+=1
    else:
        array.append(False)
        i+=1

### Making the array 'array' a numpy array 
* Making the numpy array into a dataframe.

In [6]:
array = np.array(array)
Join1314 = Join1314[array]

### Creating a dataframe ERROR with all players that have problems
* Contains players in Join that are duplicated using 'Players' or 'full_name'.

In [7]:
Error1314 = Join1314[(Join1314['Players'].isin(Join1314['Players'][Join1314['Players'].duplicated()]) | Join1314['full_name'].isin(Join1314['full_name'][Join1314['full_name'].duplicated()]))]
Error1314.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
13,192985,K. De Bruyne,Kevin De Bruyne,Manchester City,https://cdn.sofifa.org/18/teams/10.png,2162,26,English Premier League,1991-06-28,181.0,...,1,2,0,0,0,0,0,3,2.333333,Kevin
14,192985,K. De Bruyne,Kevin De Bruyne,Manchester City,https://cdn.sofifa.org/18/teams/10.png,2162,26,English Premier League,1991-06-28,181.0,...,0,6,0,0,0,0,0,16,3.875000,Kevin
29,176635,M. Ozil,Mesut Ozil,Arsenal,https://cdn.sofifa.org/18/teams/1.png,1927,28,English Premier League,1988-10-15,180.0,...,1,11,0,0,0,0,0,26,4.076923,Mesut
30,176635,M. Ozil,Mesut Ozil,Arsenal,https://cdn.sofifa.org/18/teams/1.png,1927,28,English Premier League,1988-10-15,180.0,...,0,2,0,0,0,0,0,2,2.000000,Mesut
44,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,0,1,0,0,44,22,0,24,1.083333,Marcelo


### Creating a dataframe TopLeagues with all players inside the Top 5 leagues
* Takes all players in ERROR insode the Top 5 leagues using str.contains.

In [8]:
leagues = ['German Bundesliga','Spanish Primera Division','English Premier League','French Ligue 1','Italian Serie A']
TopLeagues1314 = Error1314[(Error1314.league.str.contains('|'.join(leagues)) == True) | Error1314['league'].isnull()]

### Creating a dataframe DELETE with all players outside the Top 5 leagues
* Takes all players in ERROR outside the Top 5 leagues using str.contains.

In [9]:
Delete1314 = Error1314[Error1314.league.str.contains('|'.join(leagues)) == False]

### Creating a dataframe FIX
* This dataframe takes all players in the Top 5 that have been duplicated .

In [10]:
Fix1314 = TopLeagues1314[(TopLeagues1314['Players'].isin(TopLeagues1314['Players'][TopLeagues1314['Players'].duplicated()]) | TopLeagues1314['full_name'].isin(TopLeagues1314['full_name'][TopLeagues1314['full_name'].duplicated()]))]
Fix1314.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
13,192985,K. De Bruyne,Kevin De Bruyne,Manchester City,https://cdn.sofifa.org/18/teams/10.png,2162,26,English Premier League,1991-06-28,181.0,...,1,2,0,0,0,0,0,3,2.333333,Kevin
14,192985,K. De Bruyne,Kevin De Bruyne,Manchester City,https://cdn.sofifa.org/18/teams/10.png,2162,26,English Premier League,1991-06-28,181.0,...,0,6,0,0,0,0,0,16,3.875000,Kevin
29,176635,M. Ozil,Mesut Ozil,Arsenal,https://cdn.sofifa.org/18/teams/1.png,1927,28,English Premier League,1988-10-15,180.0,...,1,11,0,0,0,0,0,26,4.076923,Mesut
30,176635,M. Ozil,Mesut Ozil,Arsenal,https://cdn.sofifa.org/18/teams/1.png,1927,28,English Premier League,1988-10-15,180.0,...,0,2,0,0,0,0,0,2,2.000000,Mesut
44,176676,Marcelo,Marcelo Vieira da Silva,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2214,29,Spanish Primera Division,1988-05-12,174.0,...,0,1,0,0,44,22,0,24,1.083333,Marcelo


### Removing players from JOIN
* Removing players whos full name is in Delete and have the same birth date.
* Removing players whos full name is in Fix and have the same birth date.

In [11]:
Join1314 = Join1314[(~(Join1314.full_name.isin(Delete1314.full_name) & Join1314.birth_date.isin(Delete1314.birth_date)))]
Join1314 = Join1314[(~(Join1314.full_name.isin(Fix1314.full_name) & Join1314.birth_date.isin(Fix1314.birth_date)))]

### Removing players from FIX
* Removing players thats 'full_name' is not equal to 'Players'.
* Dropping all duplicate players in Fixed.

In [12]:
Fixed1314 = Fix1314[Fix1314['full_name'] == Fix1314['Players']]
Fixed1314 = Fixed1314[(Fixed1314['Players'].isin(Fixed1314['Players'][Fixed1314['Players'].duplicated()]) | Fixed1314['full_name'].isin(Fixed1314['full_name'][Fixed1314['full_name'].duplicated()]))]
Fixed1314 = Fixed1314[~Fixed1314.duplicated(['Players', 'Clubs', 'Points'])]
Fixed1314.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
13,192985,K. De Bruyne,Kevin De Bruyne,Manchester City,https://cdn.sofifa.org/18/teams/10.png,2162,26,English Premier League,1991-06-28,181.0,...,1,2,0,0,0,0,0,3,2.333333,Kevin
14,192985,K. De Bruyne,Kevin De Bruyne,Manchester City,https://cdn.sofifa.org/18/teams/10.png,2162,26,English Premier League,1991-06-28,181.0,...,0,6,0,0,0,0,0,16,3.875000,Kevin
29,176635,M. Ozil,Mesut Ozil,Arsenal,https://cdn.sofifa.org/18/teams/1.png,1927,28,English Premier League,1988-10-15,180.0,...,1,11,0,0,0,0,0,26,4.076923,Mesut
30,176635,M. Ozil,Mesut Ozil,Arsenal,https://cdn.sofifa.org/18/teams/1.png,1927,28,English Premier League,1988-10-15,180.0,...,0,2,0,0,0,0,0,2,2.000000,Mesut
198,162240,M. Dembele,Moussa Dembele,Tottenham Hotspur,https://cdn.sofifa.org/18/teams/18.png,2186,29,English Premier League,1987-07-16,185.0,...,6,11,0,0,0,0,0,28,1.785714,Moussa


### Adding players who have transfered teams
* Get the sum of all relevant columns to get one full player

In [13]:
d = {
 'ID' : 'first',
 'club_logo':'first',
 'special':'first',
 'age':'first',
 'league':'first',
 'birth_date':'first',
 'height_cm':'first',
 'weight_kg':'first',
 'body_type':'first',
 'real_face':'first',
 'flag':'first',
 'nationality':'first',
 'photo':'first',
 'eur_value':'first',
 'eur_wage':'first',
 'eur_release_clause':'first',
 'overall':'first',
 'potential':'first',
 'pac':'first',
 'sho':'first',
 'pas':'first',
 'dri':'first',
 'def':'first',
 'phy':'first',
 'international_reputation':'first',
 'skill_moves':'first',
 'weak_foot':'first',
 'work_rate_att':'first',
 'work_rate_def':'first',
 'preferred_foot':'first',
 'crossing':'first',
 'finishing':'first',
 'heading_accuracy':'first',
 'short_passing':'first',
 'volleys':'first',
 'dribbling':'first',
 'curve':'first',
 'free_kick_accuracy':'first',
 'long_passing':'first',
 'ball_control':'first',
 'acceleration':'first',
 'sprint_speed':'first',
 'agility':'first',
 'reactions':'first',
 'balance':'first',
 'shot_power':'first',
 'jumping':'first',
 'stamina':'first',
 'strength':'first',
 'long_shots':'first',
 'aggression':'first',
 'interceptions':'first',
 'positioning':'first',
 'vision':'first',
 'penalties':'first',
 'composure':'first',
 'marking':'first',
 'standing_tackle':'first',
 'sliding_tackle':'first',
 'gk_diving':'first',
 'gk_handling':'first',
 'gk_kicking':'first',
 'gk_positioning':'first',
 'gk_reflexes':'first',
 'rs':'first',
 'rw':'first',
 'rf':'first',
 'ram':'first',
 'rcm':'first',
 'rm':'first',
 'rdm':'first',
 'rcb':'first',
 'rb':'first',
 'rwb':'first',
 'st':'first',
 'lw':'first',
 'cf':'first',
 'cam':'first',
 'cm':'first',
 'lm':'first',
 'cdm':'first',
 'cb':'first',
 'lb':'first',
 'lwb':'first',
 'ls':'first',
 'lf':'first',
 'lam':'first',
 'lcm':'first',
 'ldm':'first',
 'lcb':'first',
 'gk':'first',
 '1_on_1_rush_trait':'first',
 'acrobatic_clearance_trait':'first',
 'argues_with_officials_trait':'first',
 'avoids_using_weaker_foot_trait':'first',
 'backs_into_player_trait':'first',
 'bicycle_kicks_trait':'first',
 'cautious_with_crosses_trait':'first',
 'chip_shot_trait':'first',
 'chipped_penalty_trait':'first',
 'comes_for_crosses_trait':'first',
 'corner_specialist_trait':'first',
 'diver_trait':'first',
 'dives_into_tackles_trait':'first',
 'diving_header_trait':'first',
 'driven_pass_trait':'first',
 'early_crosser_trait':'first',
 "fan's_favourite_trait":'first',
 'fancy_flicks_trait':'first',
 'finesse_shot_trait':'first',
 'flair_trait':'first',
 'flair_passes_trait':'first',
 'gk_flat_kick_trait':'first',
 'gk_long_throw_trait':'first',
 'gk_up_for_corners_trait':'first',
 'giant_throw_in_trait':'first',
 'inflexible_trait':'first',
 'injury_free_trait':'first',
 'injury_prone_trait':'first',
 'leadership_trait':'first',
 'long_passer_trait':'first',
 'long_shot_taker_trait':'first',
 'long_throw_in_trait':'first',
 'one_club_player_trait':'first',
 'outside_foot_shot_trait':'first',
 'playmaker_trait':'first',
 'power_free_kick_trait':'first',
 'power_header_trait':'first',
 'puncher_trait':'first',
 'rushes_out_of_goal_trait':'first',
 'saves_with_feet_trait':'first',
 'second_wind_trait':'first',
 'selfish_trait':'first',
 'skilled_dribbling_trait':'first',
 'stutter_penalty_trait':'first',
 'swerve_pass_trait':'first',
 'takes_finesse_free_kicks_trait':'first',
 'target_forward_trait':'first',
 'team_player_trait':'first',
 'technical_dribbler_trait':'first',
 'tries_to_beat_defensive_line_trait':'first',
 'poacher_speciality':'first',
 'speedster_speciality':'first',
 'aerial_threat_speciality':'first',
 'dribbler_speciality':'first',
 'playmaker_speciality':'first',
 'engine_speciality':'first',
 'distance_shooter_speciality':'first',
 'crosser_speciality':'first',
 'free_kick_specialist_speciality':'first',
 'tackling_speciality':'first',
 'tactician_speciality':'first',
 'acrobat_speciality':'first',
 'strength_speciality':'first',
 'clinical_finisher_speciality':'first',
 'prefers_rs':'first',
 'prefers_rw':'first',
 'prefers_rf':'first',
 'prefers_ram':'first',
 'prefers_rcm':'first',
 'prefers_rm':'first',
 'prefers_rdm':'first',
 'prefers_rcb':'first',
 'prefers_rb':'first',
 'prefers_rwb':'first',
 'prefers_st':'first',
 'prefers_lw':'first',
 'prefers_cf':'first',
 'prefers_cam':'first',
 'prefers_cm':'first',
 'prefers_lm':'first',
 'prefers_cdm':'first',
 'prefers_cb':'first',
 'prefers_lb':'first',
 'prefers_lwb':'first',
 'prefers_ls':'first',
 'prefers_lf':'first',
 'prefers_lam':'first',
 'prefers_lcm':'first',
 'prefers_ldm':'first',
 'prefers_lcb':'first',
 'prefers_gk':'first',
 'Players':'first',
 'Position':'first',
 'Clubs':'first',
 'Points':'sum',  
 'Start':'sum', 
 'Sub':'sum', 
 'club':'last',
 'Position':'first',
 'Start':'sum',
 'Goals':'sum',
 'A':'sum',
 'CS':'sum',
 'CS part':'sum',
 'Yellow':'sum',
 'Red':'sum',
 'Sub':'sum',
 'CO':'sum',
 'Off':'sum',
 'Pen SV':'sum',
 'Pen M':'sum',
 'Goals conceded':'sum',
 'Conceded 1+':'sum',
 'OG':'sum',
 'Firstname':'first',
 'Apps':'sum',
 'Form': 'mean'
}

#### Groupby function applying 'd' to fix joining the players.

In [14]:
Fixed1314 = (Fixed1314.groupby(['name','full_name'], sort=False, as_index=False).agg(d).reindex(columns=Fix1314.columns))
Fixed1314.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,192985,K. De Bruyne,Kevin De Bruyne,Manchester City,https://cdn.sofifa.org/18/teams/10.png,2162,26,English Premier League,1991-06-28,181.0,...,1,8,0,0,0,0,0,19,3.104167,Kevin
1,176635,M. Ozil,Mesut Ozil,Arsenal,https://cdn.sofifa.org/18/teams/1.png,1927,28,English Premier League,1988-10-15,180.0,...,1,13,0,0,0,0,0,28,3.038462,Mesut
2,162240,M. Dembele,Moussa Dembele,Tottenham Hotspur,https://cdn.sofifa.org/18/teams/18.png,2186,29,English Premier League,1987-07-16,185.0,...,7,12,0,0,0,0,0,30,1.642857,Moussa
3,184200,M. Arnautovic,Marko Arnautovic,West Ham United,https://cdn.sofifa.org/18/teams/19.png,2094,28,English Premier League,1989-04-19,192.0,...,4,14,0,0,0,0,0,32,2.383333,Marko
4,167431,G. Castro,Gonzalo Castro,Borussia Dortmund,https://cdn.sofifa.org/18/teams/22.png,2156,30,German Bundesliga,1987-06-11,172.0,...,0,11,0,0,38,14,0,30,3.533333,Gonzalo


#### Quick look at what FIXED with most relevant columns

In [15]:
Fixed1314[['full_name','club','Players','Clubs','Points']].head(10)

,full_name,club,Players,Clubs,Points
0,Kevin De Bruyne,Manchester City,Kevin De Bruyne,CHEL,69
1,Mesut Ozil,Arsenal,Mesut Ozil,ARSL,110
2,Moussa Dembele,Tottenham Hotspur,Moussa Dembele,SPURS,53
3,Marko Arnautovic,West Ham United,Marko Arnautovic,STOKEC,101
4,Gonzalo Castro,Borussia Dortmund,Gonzalo Castro,BL1,106
5,Mamadou Sakho,Crystal Palace,Mamadou Sakho,LPOOL,32
6,Adil Rami,Olympique de Marseille,Adil Rami,VALA,76
7,Andres Guardado,Real Betis Balompie,Andres Guardado,BL1,30
8,Faouzi Ghoulam,Napoli,Faouzi Ghoulam,NAPO,118
9,Josuha Guilavogui,VfL Wolfsburg,Josuha Guilavogui,ASSE,19


### Joining Join and Fixed
* Using pd.concat to join the two dataframes together.
* Drop irrelevant column.

In [16]:
Joined1314 = pd.concat([Join1314,Fixed1314])
Joined1314 = Joined1314.reset_index()
Joined1314.drop(Joined1314.columns[[0]], axis=1, inplace=True)
Joined1314.head(5)

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,CO,Off,Pen SV,Pen M,Goals conceded,Conceded 1+,OG,Apps,Form,Firstname
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera Division,1985-02-05,185.0,...,0,3,0,0,0,0,0,30,7.733333,Cristiano
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera Division,1987-06-24,170.0,...,2,4,0,1,0,0,0,31,7.032258,Lionel
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,...,7,11,0,0,0,0,0,26,4.153846,Neymar
3,176580,L. Suarez,Luis Suarez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera Division,1987-01-24,182.0,...,0,3,0,0,0,0,0,33,7.696970,Luis
4,167495,M. Neuer,Manuel Neuer,FC Bayern Munich,https://cdn.sofifa.org/18/teams/21.png,1493,31,German Bundesliga,1986-03-27,193.0,...,0,1,0,0,20,4,0,31,3.806452,Manuel


#### Saving to csv file in data/prep

In [17]:
Joined1314.to_csv('../../data/prep/Joined1314.csv')